In [1]:
import pickle
import numpy as np
from collections import Counter

In [2]:
number_of_games = 100000
filename = "/home/jaoi/master22/pet_for_sale/winning_games_db/4PPO_{}_games.pkl".format(number_of_games)
# with open("/home/jaoi/master22/pet_for_sale/winning_games_db/{}_games.pkl".format(number_of_games), "rb") as fp:
with open(filename, "rb") as fp:
        games = pickle.load(fp)

In [23]:
def cum_enc(num, max_num):
    enc = np.zeros(max_num)
    for i in range(int(num)):
        enc[i] = 1.0
    return enc

In [112]:
values = list(range(1, 19))

15

In [97]:
def convert_4_rounds_fs1_obs_to_cum_bin_obs(obs):
    # 4 rounds, so hands also need to have 4 cards ... 
    stack_len = 15
    rounds = 4
    # Should be 9
    coin_numbers = [0, 6, 7, 10, 16, 17, 20, 26, 27]
    
    # board_card_numbers = [1,2,3,4,8,11,12,13,14,18,21,22,23,24,28,45,46,47]
    board_numbers = [45,46,47]
    can_bet_numbers = [5,15,25]
    order_numbers = [9,19,29]
    
    std_number = 48
    stack_numbers = list(range(30, 45))
    bin_rep = []
    onehot_of_stack = np.zeros(stack_len, dtype="float32")
    
    p1_cards = [1,2,3,4]
    p1_coins = 0
    p2_cards = [11,12,13,14]
    p2_coins = 10
    p3_cards = [21,22,23,24]
    p3_coins = 20
    
    p_values = {}
    p_values[0] = sum([obs[card] for card in p1_cards]) + obs[p1_coins]
    p_values[10] = sum([obs[card] for card in p2_cards]) + obs[p2_coins]
    p_values[20] = sum([obs[card] for card in p3_cards]) + obs[p3_coins]
    
    
    player_bid_numbers = [6, 16, 26]
    p_values[player_bid_numbers[0]] = sum([obs[card] for card in p1_cards]) + obs[p1_coins] - np.ceil(obs[player_bid_numbers[0]]/2) + obs[47]
    p_values[player_bid_numbers[1]] = sum([obs[card] for card in p2_cards]) + obs[p2_coins] - np.ceil(obs[player_bid_numbers[1]]/2) + obs[46]
    p_values[player_bid_numbers[2]] = sum([obs[card] for card in p3_cards]) + obs[p3_coins] - obs[player_bid_numbers[2]] + obs[45]
    
    player_coin_numbers = [0, 10, 20]
    
    
    for i, num in enumerate(obs):
        if i in player_coin_numbers:
            bin_rep.extend(cum_enc(num, 18))
            # extending the cum encoding of current score
            bin_rep.extend(cum_enc(p_values[i], 62))
        elif i in can_bet_numbers:
            bin_rep.append(num)
        elif i in player_bid_numbers:
            bin_rep.extend(cum_enc(num, 18))
            bin_rep.extend(cum_enc(p_values[i], 62))
        elif i in order_numbers:
            order = np.zeros(3)
            order[int(num)-1] = 1.0
            bin_rep.extend(order)
        elif i in board_numbers:
            bin_rep.extend(cum_enc(num, 15))
        elif i in stack_numbers:
            if i == stack_numbers[-1]:
                if int(num) != 0:
                    onehot_of_stack[int(num) - 1] = 1
                    bin_rep.extend(list(onehot_of_stack))
                else:
                    # onehot_of_stack[-int(num)] = 1
                    bin_rep.extend(list(onehot_of_stack))
            else:
                if int(num) != 0:
                    onehot_of_stack[int(num)- 1] = 1
        elif i == std_number:
            if num > (np.std([1,stack_len]) - np.std([1, 2])) / 2:
                bin_rep.append(1)
            else:
                bin_rep.append(0)
        # print(p_values[player_bid_numbers[0]], p_values[player_bid_numbers[1]], p_values[player_bid_numbers[2]])
        
        
       
        
  
    
    # print(obs)
    # print(p_values[0], p_values[10], p_values[20])
    # print(obs[6]/2, obs[16]/2, obs[26])
    # print(obs[47], obs[46], obs[45])
    # print(p1_score_on_fold, p2_score_on_fold, p3_score_on_fold)

    # print(onehot_of_stack)
    # print(obs[30:45], len(obs[30:45]))
    bin_rep = np.hstack(bin_rep)
    return bin_rep.astype(np.float32)

In [101]:
(18 + 62 + 18 + 62 + 3 + 1) * 3 + 3 * 15 + 15 + 1 

553

In [104]:
18+62+18+62+3+1-1

163

In [91]:
def find_highest_total_score(games):
    highest = 49
    
    p1_cards = [1,2,3,4,]
    p1_coins = 0
    p2_cards = [11,12,13,14]
    p2_coins = 10
    p3_cards = [21,22,23,24]
    p3_coins = 20
    for game in games:
        p1_score = sum([game[0][card] for card in p1_cards]) + game[0][p1_coins]
        p2_score = sum([game[0][card] for card in p2_cards]) + game[0][p2_coins]
        p3_score = sum([game[0][card] for card in p3_cards]) + game[0][p3_coins]
    
        if p1_score > highest:
            highest = p1_score
            print(highest)
        if p2_score > highest:
            highest = p2_score
            print(highest)
        if p3_score > highest:
            highest = p3_score
            print(highest)
 
    
find_highest_total_score(games)

50.0


In [98]:
convert_4_rounds_fs1_obs_to_cum_bin_obs(games[0][0]).shape


(553,)

In [99]:
c = Counter()
tsetlinified_games = []
for i, game in enumerate(games):
    tset_game = convert_4_rounds_fs1_obs_to_cum_bin_obs(game[0])
    c[tset_game.shape] += 1
    
    tsetlinified_games.append((tset_game, game[1]))


print(c)
tsetlinified_games = np.array(tsetlinified_games, dtype=object)


Counter({(553,): 707104})


In [100]:
tsetlin_filename = "/home/jaoi/master22/pet_for_sale/winning_games_db/4PPO_{}_cum_tsetlined_games.pkl".format(number_of_games)
with open(tsetlin_filename, "wb") as fp:
        pickle.dump(tsetlinified_games, fp)